In [1]:
# pip install transformers

In [2]:
# pip install torch

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import torch
#using distillbert
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
import torch.nn as nn
import torch.optim as optim
# jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [4]:
df_original = pd.read_csv(r"/Users/chandanshashwat/Desktop/DataSet/twitterS.csv")

In [14]:
df = df_original[:5000].copy()

In [15]:
df.head()

,Sentiment,Text
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


In [16]:
df.shape

(5000, 2)

In [17]:
df["Sentiment"].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [18]:
df["Sentiment"].value_counts()

Positive      1938
Neutral       1166
Negative      1056
Irrelevant     840
Name: Sentiment, dtype: int64

In [19]:
df['Sentiment']= df['Sentiment'].replace(['Positive', 'Neutral', 'Negative', 'Irrelevant'], [0,1,2,3])

In [36]:
df['Sentiment'].unique()

array([0, 1, 2, 3])

In [29]:
df.dropna(inplace=True)
df.isnull().sum()

Sentiment    0
Text         0
dtype: int64

In [30]:
X = list(df["Text"].values)
y = list(df["Sentiment"].values)

from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    X, y, random_state=42, test_size=0.2,
)

In [31]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [32]:
class hamspamdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = hamspamdataset(train_encodings, train_labels)
test_dataset = hamspamdataset(test_encodings, test_labels)

In [33]:
# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
#     acc = accuracy_score(labels, preds)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }

In [39]:
training_args = TrainingArguments(
    output_dir="results",  # output directory
    overwrite_output_dir=True,
    num_train_epochs=3,  # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,  # batch size for evaluation
    warmup_steps=1000,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir="logs",  # directory for storing logs
    logging_steps=5000,  # default: 500
    save_steps=5000,  # default: 500
    learning_rate=1e-5,
    do_train=True,
    do_eval=True,
    seed=16,
    gradient_accumulation_steps=8,  # reduce memory usage while allowing bigger overall batch size.
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

trainer = Trainer(
    model=model,  # the instantiated Transformers model to be trained
    args=training_args,  # training arguments, defined above
#     compute_metrics=compute_metrics,
    train_dataset=train_dataset,  # training dataset
    eval_dataset=test_dataset,  # test dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /Users/chandanshashwat/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,

In [40]:
trainer.train()

***** Running training *****
  Num examples = 3961
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 369
  Number of trainable parameters = 66956548


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=369, training_loss=1.3326665780085536, metrics={'train_runtime': 2269.9504, 'train_samples_per_second': 5.235, 'train_steps_per_second': 0.163, 'total_flos': 699521117448384.0, 'train_loss': 1.3326665780085536, 'epoch': 2.99})

In [42]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 991
  Batch size = 4


{'eval_loss': 1.1703726053237915,
 'eval_runtime': 44.2624,
 'eval_samples_per_second': 22.389,
 'eval_steps_per_second': 5.603,
 'epoch': 2.99}

In [43]:
ft_model = "finetuned/twitter_sentim"
trainer.save_model(ft_model)
# tokenizer.save_pretrained(ft_model)
# output_dir = './model_save/'
# model_to_save.save_pretrained(output_dir)

Saving model checkpoint to finetuned/twitter_sentim
Configuration saved in finetuned/twitter_sentim/config.json
Model weights saved in finetuned/twitter_sentim/pytorch_model.bin


In [46]:
ft_tokanizer = "finetuned/tokenizer"
tokenizer.save_pretrained(ft_tokanizer)

tokenizer config file saved in finetuned/tokenizer/tokenizer_config.json
Special tokens file saved in finetuned/tokenizer/special_tokens_map.json


('finetuned/tokenizer/tokenizer_config.json',
 'finetuned/tokenizer/special_tokens_map.json',
 'finetuned/tokenizer/vocab.txt',
 'finetuned/tokenizer/added_tokens.json',
 'finetuned/tokenizer/tokenizer.json')

In [45]:
tokenizer("chandandodo")

{'input_ids': [101, 9212, 7847, 3527, 3527, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [56]:
#Prediction 
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 991
  Batch size = 4


PredictionOutput(predictions=array([[ 0.92060155,  0.01549772, -0.48461488, -0.48578054],
       [ 0.773078  ,  0.08366629, -0.40965343, -0.38616055],
       [ 0.5110557 , -0.225641  , -0.07265653, -0.5824527 ],
       ...,
       [ 0.78424215, -0.30756932, -0.53917176, -0.20832232],
       [ 0.6231303 ,  0.12010843, -0.56717354, -0.115559  ],
       [ 0.91952723, -0.19310656, -0.5575187 , -0.49310625]],
      dtype=float32), label_ids=array([0, 0, 2, 0, 0, 2, 1, 2, 2, 0, 1, 0, 0, 1, 1, 1, 3, 1, 0, 2, 2, 1,
       3, 2, 0, 3, 1, 1, 0, 1, 1, 2, 1, 1, 1, 3, 0, 3, 3, 0, 0, 0, 1, 1,
       0, 0, 0, 3, 1, 2, 0, 1, 2, 2, 3, 3, 0, 1, 2, 0, 0, 0, 1, 0, 2, 0,
       1, 0, 0, 2, 3, 1, 3, 3, 0, 0, 0, 0, 3, 0, 2, 0, 0, 2, 1, 0, 0, 3,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 2, 0, 0, 1, 1,
       2, 0, 0, 3, 3, 0, 3, 1, 1, 3, 0, 0, 2, 3, 2, 0, 1, 1, 1, 0, 2, 0,
       0, 0, 1, 1, 0, 3, 0, 0, 0, 0, 0, 2, 0, 1, 3, 3, 0, 2, 0, 2, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 3, 0, 0, 0, 0, 0, 1, 

In [60]:
model_dist = DistilBertForSequenceClassification.from_pretrained('./finetuned/twitter_sentim/',local_files_only=True)

loading configuration file ./finetuned/twitter_sentim/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file ./finetuned/twitter_sentim/pytorch_model.bin
All model checkpoint weights w

In [87]:
# with torch.no_grad():
#     logits = model_dist(**inputs).logits

# inputs = tokenizer(df_train[0], return_tensors="pt")

# predicted_class_id = logits.argmax().item()
# model.config.id2label[predicted_class_id]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
text = ["no"]
encoding = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)
outputs = model_dist(**encoding)
predictions = outputs.logits.argmax(-1)
predictions

tensor([0])

In [91]:
# inputs_prediction = tokenizer(df_train[0], return_tensors="pt")
# outputs = model_dist(**inputs_prediction)

# logits = outputs.logits
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model_dist, tokenizer=tokenizer)
prediction = pipe("The text to predict", return_all_scores=True)
prediction
#check in config.json file the Labels defined

[[{'label': 'LABEL_0', 'score': 0.39412593841552734},
  {'label': 'LABEL_1', 'score': 0.1777602583169937},
  {'label': 'LABEL_2', 'score': 0.25337693095207214},
  {'label': 'LABEL_3', 'score': 0.17473682761192322}]]

In [ ]:
logits = outputs.logits
predicted_label_classes = logits.argmax(-1)
print(predicted_label_classes)